In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import json

driver = webdriver.Chrome()

# Twitter的URL
twitter_url = 'https://twitter.com/'

# 从文件加载cookies
cookies_path = r'\twitter_Crawler\twitter_cookie.json'
with open(cookies_path, 'r', encoding='utf-8') as cookies_file:
    cookies = json.load(cookies_file)

# 访问Twitter
driver.get(twitter_url)

# 等待页面加载完成
time.sleep(5)

# 添加cookie到浏览器
for cookie in cookies:
    driver.add_cookie(cookie)

# 再次访问Twitter，这次将会使用加载的cookies
driver.get(twitter_url)

# Twitter搜索URL
twitter_search_url = '替换搜索结果连接，如要爬取"Python"相关的推文，在Twitter搜索"Python"，然后复制URL到这里'

# 导航到Twitter搜索页面
driver.get(twitter_search_url)
time.sleep(5)  # 给页面时间加载

# 用于存储抓取到的数据
tweets_data = []

# 设置要滚动的次数
scrolls = 3
for i in range(scrolls):
    # 找到所有的推文元素
    tweets = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')
    for tweet in tweets:
        # 提取用户名
        user_name = tweet.find_element(By.XPATH, './/span').text

        # 提取文本内容
        tweet_text = tweet.find_element(By.XPATH, './/div[2]/div[2]/div[2]/div[1]').text

        # 提取图片链接（如果有的话）
        image_url = None
        try:
            image_element = tweet.find_element(By.XPATH, './/img[@alt="图像"]')
            image_url = image_element.get_attribute('src')
        except Exception as e:
            pass  # 如果没有找到图片，则跳过

        # 存储数据
        tweets_data.append({
            'user_name': user_name,
            'text': tweet_text,
            'image_url': image_url,
        })

    # 滚动到页面底部
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(2)  # 等待页面加载新的推文

# 将数据保存为JSON文件
with open('tweets_data.json', 'w', encoding='utf-8') as f:
    json.dump(tweets_data, f, ensure_ascii=False, indent=4)

# 完成后关闭浏览器
driver.quit()
